In [ ]:
import os, json, glob, sys, math, numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import time
from time import strftime

from tensorflow.keras.metrics import BinaryAccuracy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
tf.enable_eager_execution()

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # gpu number

config = tf.ConfigProto()

# use 30% memory of gpu
config.gpu_options.per_process_gpu_memory_fraction = 0.3
session = tf.Session(config=config)

In [ ]:
* gpu 확인

In [ ]:
tf.config.experimental_list_devices()

In [ ]:
* 자료형 선언

In [ ]:
tf.keras.backend.set_floatx('float32')

In [ ]:
* load data (use built-in data of breast cancer)

In [ ]:
from sklearn.datasets import load_breast_cancer

scaler = MinMaxScaler()
file = load_breast_cancer()
X, Y = file['data'], file['target']
X = scaler.fit_transform(X)

In [ ]:
n = X.shape[0]
p = X.shape[1]
k = 10
batch_size = 8
epochs = 50

In [ ]:
file.keys()

print(file['data'].shape)
print(file['target'].shape)

In [ ]:
* FM Model

In [ ]:
class FM(tf.keras.Model):
    def __init__(self):
        super(FM, self).__init__()
        
        self.w_0 = tf.Variable([0.0])
        self.w = tf.Variable(tf.zeros([p]))
        self.V = tf.Variable(tf.random.normal(shape=(p, k)))
        
    def call(self, inputs):
        # p != n
        # n : the num. of data
        # p : the num. of features?
        # w : 1 x p
        # inputs : 1 or n x p
        # V : p x k
        
        linear_terms = tf.reduce_sum(tf.math.multiply(self.w, inputs), axis=1)
        
        interactions = 0.5 * tf.reduce_sum(
            tf.math.pow(tf.matmul(inputs, self.V), 2) # v * x 가 n^2개 있는데, 그걸 각각 제곱.
            - tf.matmul(tf.math.pow(inputs, 2), tf.math.pow(self.V, 2)),
        1,
        keepdims=False
        )
        
        y_hat = tf.math.sigmoid(self.w_0 + linear_terms + interactions)
        
        return y_hat

In [ ]:
* Learning

In [ ]:
def train_on_batch(model, optimizer, accuracy, inputs, targets):
    with tf.GradientTape() as tape:
        y_pred = model(inputs)
        loss = tf.keras.losses.binary_crossentropy(from_logits=False,
                                                  y_true=targets,
                                                  y_pred=y_pred)
        
        grads = tape.gradient(target=loss, sources=model.trainable_variables)
        
        # apply_gradients()를 통해 processed gradients 적용 (optimization)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        # update 마다 accuracy 누적하여 계산. (loss와 별개로..? 계산)
        accuracy.update_state(targets, y_pred)
        
        return loss
    
def train(epochs):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)
    
    train_ds = tf.data.Dataset.from_tensor_slices(
    (tf.cast(X_train, tf.float32), tf.cast(Y_train, tf.float32))).shuffle(500).batch(8)

    test_ds = tf.data.Dataset.from_tensor_slices(
    (tf.cast(X_test, tf.float32), tf.cast(Y_test, tf.float32))).shuffle(200).batch(8)
    
    model = FM()
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    accuracy = BinaryAccuracy(threshold=0.5)
    loss_history = []
    
    for i in range(epochs):
        for x, y in train_ds:
            loss = train_on_batch(model, optimizer, accuracy, x, y)
            loss_history.append(loss)
            
        if i % 2 == 0:
            print("Step {:03d}, Cumulative mean loss: {:.4f}, Cumulative accuracy: {:.4f}".format(i, np.mean(loss_history), accuracy.result().numpy()))
            
    test_accuracy = BinaryAccuracy(threshold=0.5)
    for x, y in test_ds:
        y_pred = model(x)
        test_accuracy.update_state(y, y_pred)
        
    print("Test Accuracy: {:.4f}".format(test_accuracy.result().numpy()))

In [ ]:
train(epochs)